In [ ]:
import numpy as np
import pandas as pd

import skimage
import skimage.io
import skimage.filters
import skimage.morphology
from skimage.filters import threshold_otsu
from skimage import registration
from skimage.feature import ORB, match_descriptors
from skimage.transform import matrix_transform
from skimage import data
from skimage.feature import register_translation
from skimage.feature.register_translation import _upsampled_dft
from skimage.morphology import watershed
from skimage.feature import peak_local_max
from skimage.measure import label
from skimage.morphology import closing, square
from skimage.measure import regionprops
from skimage.color import label2rgb

from scipy import ndimage as ndi
from scipy.ndimage import fourier_shift

from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from holoviews.operation import decimate
from holoviews.plotting.util import process_cmap

import os
import glob

import bebi103

import colorcet

import bokeh
bokeh.io.output_notebook()

import holoviews as hv
hv.extension('bokeh')
bebi103.hv.set_defaults()

import panel as pn

from sklearn import linear_model

In [ ]:
# The directory containing the images
data_dir = '../data/200129_double_elect_spinowitz'

# glob string for images
im_glob = os.path.join(data_dir, '*')

# Get list of images
im_list = sorted(glob.glob(im_glob))

im_list

In [ ]:
dapi_raw = skimage.io.imread(im_list[0])
cfp_raw = skimage.io.imread(im_list[1])
chfp_raw = skimage.io.imread(im_list[2])

In [ ]:
# equivalent to matlab imadjust, can set bounds
def imadjust(img, lower_bound=0.25, upper_bound=99.75):
    lower = np.percentile(img, lower_bound)
    upper = np.percentile(img, upper_bound)
    out = (img - lower) * (255 / (upper - lower))
    return np.clip(out, 0, 255, out)

dapi_adj = imadjust(dapi_raw)
cfp_adj = imadjust(cfp_raw)
chfp_adj = imadjust(chfp_raw)

In [ ]:
'''
# Define slider for slice within z-stack
slice_slider = pn.widgets.FloatSlider(
    name = 'slice',
    start=0,
    end=30,
    step=1,
    value=1)

@pn.depends(slice_slider.param.value)
def im_stack(slice):
    return (hv.Layout(hv.Image(dapi_adj[slice]).opts(title='dapi')
                      +hv.Image(cfp_adj[slice]).opts(title='cfp')
                      +hv.Image(chfp_adj[slice]).opts(title='chfp')
                     ).cols(2))

pn.Row(
    im_stack,
    pn.Column(slice_slider), width_policy='min')
'''

I will use slice 1 as a test.

In [ ]:
# Take a slice
dapi = dapi_adj[30]
cfp = cfp_adj[30]
chfp = chfp_adj[30]

I will now register the CFP and ChFP channels by finding the offset of ChFP, then translating/shifting the ChFP image accordingly.

In [ ]:
# pixel precision first
shift, error, diffphase = register_translation(cfp, chfp)
print(f"Detected pixel offset (y, x): {shift}")

In [ ]:
chfp_shift = ndi.shift(chfp, shift)

# corrected chfp_slice
chfp = chfp_shift

In [ ]:
# Also slice and fix the raw images
dapi_raw = dapi_raw[30]
cfp_raw = cfp_raw[30]
chfp_raw = ndi.shift(chfp_raw[30], shift)

I will define functions to display plots side by side.

In [ ]:
def show_two_ims(
    im_1,
    im_2,
    titles=[None, None],
    interpixel_distances=[0.13, 0.13],
    cmap=[None, None]
):
    """Convenient function for showing two images side by side."""
    p_1 = bebi103.image.imshow(
        im_1,
        frame_height=225,
        title=titles[0],
        cmap=cmap[0],
        #interpixel_distance=interpixel_distances[0],
        #length_units="µm",
    )
    p_2 = bebi103.image.imshow(
        im_2,
        frame_height=225,
        title=titles[1],
        cmap=cmap[1],
        #interpixel_distance=interpixel_distances[1],
        #length_units="µm",
    )
    p_2.x_range = p_1.x_range
    p_2.y_range = p_1.y_range

    return bokeh.layouts.gridplot([p_1, p_2], ncols=2)

def show_three_ims(
    im_1,
    im_2,
    im_3,
    titles=[None, None, None],
    interpixel_distances=[0.13, 0.13, 0.13],
    cmap=[None, None, None],
):
    """Convenient function for showing two images side by side."""
    p_1 = bebi103.image.imshow(
        im_1,
        frame_height=225,
        title=titles[0],
        cmap=cmap[0],
        #interpixel_distance=interpixel_distances[0],
        #length_units="µm",
    )
    p_2 = bebi103.image.imshow(
        im_2,
        frame_height=225,
        title=titles[1],
        cmap=cmap[1],
        #interpixel_distance=interpixel_distances[1],
        #length_units="µm",
    )
    p_3 = bebi103.image.imshow(
        im_3,
        frame_height=225,
        title=titles[2],
        cmap=cmap[2],
        #interpixel_distance=interpixel_distances[1],
        #length_units="µm",
    )
    p_2.x_range = p_1.x_range
    p_2.y_range = p_1.y_range
    p_3.x_range = p_1.x_range
    p_3.y_range = p_1.y_range
    
    return bokeh.layouts.gridplot([p_1, p_2, p_3], ncols=3)

def show_four_ims(
    im_1,
    im_2,
    im_3,
    im_4,
    titles=[None, None, None, None],
    interpixel_distances=[0.13, 0.13, 0.13, 0.13],
    cmap=[None, None, None, None],
):
    """Convenient function for showing two images side by side."""
    p_1 = bebi103.image.imshow(
        im_1,
        frame_height=225,
        title=titles[0],
        cmap=cmap[0],
        #interpixel_distance=interpixel_distances[0],
        #length_units="µm",
    )
    p_2 = bebi103.image.imshow(
        im_2,
        frame_height=225,
        title=titles[1],
        cmap=cmap[1],
        #interpixel_distance=interpixel_distances[1],
        #length_units="µm",
    )
    p_3 = bebi103.image.imshow(
        im_3,
        frame_height=225,
        title=titles[2],
        cmap=cmap[2],
        #interpixel_distance=interpixel_distances[1],
        #length_units="µm",
    )
    p_4 = bebi103.image.imshow(
        im_4,
        frame_height=225,
        title=titles[3],
        cmap=cmap[3],
        #interpixel_distance=interpixel_distances[1],
        #length_units="µm",
    )
    p_2.x_range = p_1.x_range
    p_2.y_range = p_1.y_range
    p_3.x_range = p_1.x_range
    p_3.y_range = p_1.y_range
    p_4.x_range = p_1.x_range
    p_4.y_range = p_1.y_range
    
    return bokeh.layouts.gridplot([p_1, p_2, p_3, p_4], ncols=2)

def show_five_ims(
    im_1,
    im_2,
    im_3,
    im_4,
    im_5,
    titles=[None, None, None, None, None],
    interpixel_distances=[0.13, 0.13, 0.13, 0.13, 0.13],
    cmap=[None, None, None, None, None],
):
    """Convenient function for showing two images side by side."""
    p_1 = bebi103.image.imshow(
        im_1,
        frame_height=225,
        title=titles[0],
        cmap=cmap[0],
        #interpixel_distance=interpixel_distances[0],
        #length_units="µm",
    )
    p_2 = bebi103.image.imshow(
        im_2,
        frame_height=225,
        title=titles[1],
        cmap=cmap[1],
        #interpixel_distance=interpixel_distances[1],
        #length_units="µm",
    )
    p_3 = bebi103.image.imshow(
        im_3,
        frame_height=225,
        title=titles[2],
        cmap=cmap[2],
        #interpixel_distance=interpixel_distances[1],
        #length_units="µm",
    )
    p_4 = bebi103.image.imshow(
        im_4,
        frame_height=225,
        title=titles[3],
        cmap=cmap[3],
        #interpixel_distance=interpixel_distances[1],
        #length_units="µm",
    )
    p_5 = bebi103.image.imshow(
        im_5,
        frame_height=225,
        title=titles[4],
        cmap=cmap[4],
        #interpixel_distance=interpixel_distances[1],
        #length_units="µm",
    )
    p_2.x_range = p_1.x_range
    p_2.y_range = p_1.y_range
    p_3.x_range = p_1.x_range
    p_3.y_range = p_1.y_range
    p_4.x_range = p_1.x_range
    p_4.y_range = p_1.y_range
    p_5.x_range = p_1.x_range
    p_5.y_range = p_1.y_range
    
    return bokeh.layouts.gridplot([p_1, p_2, p_3, p_4, p_5], ncols=3)

def show_six_ims(
    im_1,
    im_2,
    im_3,
    im_4,
    im_5,
    im_6,
    titles=[None, None, None, None, None, None],
    interpixel_distances=[0.13, 0.13, 0.13, 0.13, 0.13, 0.13],
    cmap=[None, None, None, None, None, None],
):
    """Convenient function for showing two images side by side."""
    p_1 = bebi103.image.imshow(
        im_1,
        frame_height=225,
        title=titles[0],
        cmap=cmap[0],
        #interpixel_distance=interpixel_distances[0],
        #length_units="µm",
    )
    p_2 = bebi103.image.imshow(
        im_2,
        frame_height=225,
        title=titles[1],
        cmap=cmap[1],
        #interpixel_distance=interpixel_distances[1],
        #length_units="µm",
    )
    p_3 = bebi103.image.imshow(
        im_3,
        frame_height=225,
        title=titles[2],
        cmap=cmap[2],
        #interpixel_distance=interpixel_distances[1],
        #length_units="µm",
    )
    p_4 = bebi103.image.imshow(
        im_4,
        frame_height=225,
        title=titles[3],
        cmap=cmap[3],
        #interpixel_distance=interpixel_distances[1],
        #length_units="µm",
    )
    p_5 = bebi103.image.imshow(
        im_5,
        frame_height=225,
        title=titles[4],
        cmap=cmap[4],
        #interpixel_distance=interpixel_distances[1],
        #length_units="µm",
    )
    p_6 = bebi103.image.imshow(
        im_6,
        frame_height=225,
        title=titles[5],
        cmap=cmap[5],
        #interpixel_distance=interpixel_distances[1],
        #length_units="µm",
    )
    p_2.x_range = p_1.x_range
    p_2.y_range = p_1.y_range
    p_3.x_range = p_1.x_range
    p_3.y_range = p_1.y_range
    p_4.x_range = p_1.x_range
    p_4.y_range = p_1.y_range
    p_5.x_range = p_1.x_range
    p_5.y_range = p_1.y_range
    p_6.x_range = p_1.x_range
    p_6.y_range = p_1.y_range
    
    return bokeh.layouts.gridplot([p_1, p_2, p_3, p_4, p_5, p_6], ncols=3)

In [ ]:
# Show filtered image
bokeh.io.show(
    show_three_ims(dapi, cfp, chfp, 
    titles=["DAPI", "CFP", "ChFP"]))

In [ ]:
# Show filtered image
bokeh.io.show(
    show_three_ims(dapi_raw, cfp_raw, chfp_raw, 
    titles=["DAPI", "CFP", "ChFP"]))

In [ ]:
# Make slice object
zoom1 = np.s_[600:1000, 600:1000]

# Show filtered image
bokeh.io.show(
    show_three_ims(dapi[zoom1], cfp[zoom1], chfp[zoom1], 
    titles=["DAPI", "CFP", "ChFP"]))

## Step 1. Filters

First, I will apply a gaussian filter.

In [ ]:
# Filter image w/ gaussian 
dapi_filt_gauss = skimage.filters.gaussian(dapi_raw, 1.5)
cfp_filt_gauss = skimage.filters.gaussian(cfp_raw, 1.5)
chfp_filt_gauss = skimage.filters.gaussian(chfp_raw, 1.5)

bokeh.io.show(
    show_three_ims(dapi_filt_gauss[zoom1], cfp_filt_gauss[zoom1], chfp_filt_gauss[zoom1], 
    titles=["DAPI", "CFP", "ChFP"]))

## Step 2. Thresholding

I will threshold using Otsu's method.

In [ ]:
def plot_hist(im, title, logy=False):
    """Make plot of image histogram."""
    counts, vals = skimage.exposure.histogram(im)
    if logy:
        inds = counts > 0
        log_counts = np.log(counts[inds])
        return hv.Spikes(
            data=(vals[inds], log_counts),
            kdims=['pixel values'],
            vdims=['log₁₀ count'],
            label=title,
        ).opts(
            frame_height=100,
        )

    return hv.Spikes(
        data=(vals, counts),
        kdims=['pixel values'],
        vdims=['count'],
        label=title,
    ).opts(
        frame_height=100,
    )

In [ ]:
# Display histograms
plots = [plot_hist(dapi_filt_gauss, 'dapi'),
         plot_hist(cfp_filt_gauss, 'cfp'),
         plot_hist(chfp_filt_gauss, 'chfp')
        ]
hv.Layout(
    plots
).opts(
    shared_axes=False,
).cols(
    1
)

I will use Otsu's thresholding method.

In [ ]:
threshold1 = threshold_otsu(dapi_filt_gauss)
print(threshold1, 'is where the dapi cutoff point is.')
dapi_filt_gauss_bw = dapi_filt_gauss > threshold1

threshold2 = threshold_otsu(cfp_filt_gauss)
print(threshold2, 'is where the cfp cutoff point is.')
cfp_filt_gauss_bw = cfp_filt_gauss > threshold2

threshold3 = threshold_otsu(chfp_filt_gauss)
print(threshold3, 'is where the chfp cutoff point is.')
chfp_filt_gauss_bw = chfp_filt_gauss > threshold3

# Show images
bokeh.io.show(
    show_three_ims(dapi_filt_gauss_bw, cfp_filt_gauss_bw, chfp_filt_gauss_bw,
                   titles=['dapi', 'cfp', 'chfp']))

# Show filtered image
bokeh.io.show(
    show_three_ims(dapi_filt_gauss, cfp_filt_gauss, chfp_filt_gauss, 
    titles=["DAPI", "CFP", "ChFP"]))

In [ ]:
# Show images
bokeh.io.show(
    show_three_ims(dapi_filt_gauss_bw[zoom1], cfp_filt_gauss_bw[zoom1], chfp_filt_gauss_bw[zoom1],
                   titles=['dapi', 'cfp', 'chfp']))

# Show filtered image
bokeh.io.show(
    show_three_ims(dapi_filt_gauss[zoom1], cfp_filt_gauss[zoom1], chfp_filt_gauss[zoom1], 
    titles=["DAPI", "CFP", "ChFP"]))

Now, I will dilate the image to make the nuclei more round.

In [ ]:
# Make the structuring element 1 pixel radius disk
selem = skimage.morphology.disk(0)

# Dilate image
dapi_dil = skimage.morphology.dilation(dapi_filt_gauss_bw, selem)
cfp_dil = skimage.morphology.dilation(cfp_filt_gauss_bw, selem)
chfp_dil = skimage.morphology.dilation(chfp_filt_gauss_bw, selem)
'''
# Show images
bokeh.io.show(
    show_three_ims(dapi_dil[zoom1], cfp_dil[zoom1], chfp_dil[zoom1], 
                   titles=['dapi', 'cfp', 'chfp']))
'''

It seems like the dilation process has merged some nuclei. Let's try to use the watershed tool to separate the merged nuclei. I will do this by performing distance transformation, followed by identification of local maxima. I will then use these maxima as markers, for which I will perform the watershed. The watershed will be performed on the mask from the thresholded gaussian filter image.

At this point I will do watershed for dapi first, followed by cfp and chfp. I will use watershed method 1 (peak_local_max).

## Dapi watershed

In [ ]:
distance = ndi.distance_transform_edt(dapi_dil)
local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((25,25)),
                            labels=dapi_dil)
#25 seems best for getting more single nuclei
markers = ndi.label(local_maxi)[0]
labels = watershed(-distance, markers, mask=dapi_dil)
dapi_ws_mask = skimage.morphology.remove_small_objects(labels, min_size=100)
bokeh.io.show(show_three_ims(dapi_filt_gauss, labels, dapi_ws_mask,
                             titles=['original', 'watershed', 'remove small'], 
                             cmap=[None, colorcet.b_glasbey_hv, colorcet.b_glasbey_hv]))


In [ ]:

# relabel image regions
label_image_dapi = skimage.measure.label(dapi_ws_mask)

props_dapi = skimage.measure.regionprops_table(label_image_dapi, intensity_image=dapi_raw, properties=('label',
                                                                                         'centroid',
                                                                                         'area',
                                                                                         'mean_intensity'))
df_dapi = pd.DataFrame(props_dapi)
dapi_area = df_dapi['area'].sum()
print(dapi_area)
df_dapi


## Cfp watershed

In [ ]:
distance = ndi.distance_transform_edt(cfp_dil)
local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((25, 25)),
                            labels=cfp_dil)
#25 seems best for getting more single nuclei
markers = ndi.label(local_maxi)[0]
labels = watershed(-distance, markers, mask=cfp_dil, watershed_line=True)
cfp_ws_mask = skimage.morphology.remove_small_objects(labels, min_size=50)
bokeh.io.show(show_three_ims(cfp_filt_gauss[zoom1], labels[zoom1], cfp_ws_mask[zoom1],
                             titles=['original', 'watershed', 'watershed remove small'], 
                             cmap=[None, colorcet.b_glasbey_hv, colorcet.b_glasbey_hv]))

It seems like there are some labels with very large area, probably representing multiple nuclei, I will plot a histogram of area per label to look at this more carefully.

In [ ]:
# relabel image regions
label_image_cfp = skimage.measure.label(cfp_ws_mask)

In [ ]:
props_cfp = skimage.measure.regionprops_table(label_image_cfp, intensity_image=cfp_raw, properties=('label',
                                                                                         'centroid',
                                                                                         'area',
                                                                                         'mean_intensity'))
df_cfp = pd.DataFrame(props_cfp)
cfp_area = df_cfp['area'].sum()
print(cfp_area)
df_cfp

In [ ]:
def freedman_diaconis_bins(data):
    """Number of bins based on Freedman-Diaconis rule."""
    h = 2 * (np.percentile(data, 75) - np.percentile(data, 25)) / np.cbrt(len(data))
    return int(np.ceil((data.max() - data.min()) / h))

In [ ]:
bins = freedman_diaconis_bins(df_cfp['area'])

hv.Histogram(data=np.histogram(df_cfp['area'], bins=bins),
    kdims=['area']
)


Based on this histogram, I will remove labels with area greater than 1000.

In [ ]:
cfp_array = np.zeros_like(label_image_cfp)
max_size = 1000

for i, label in enumerate(df_cfp.loc[df_cfp.loc[:, 'area'] > max_size]['label']):
    x = label_image_cfp == label
    y = x * label
    cfp_array += y

In [ ]:
cfp_large_sub = label_image_cfp - cfp_array

bokeh.io.show(
    show_three_ims(cfp_filt_gauss, cfp_ws_mask, cfp_large_sub,
                titles=['original', 'watershed', 'large subtract'],
                cmap=[None, colorcet.b_glasbey_hv, colorcet.b_glasbey_hv]))


## ChFP watershed

In [ ]:
distance = ndi.distance_transform_edt(chfp_dil)
local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((25, 25)),
                            labels=chfp_dil)
#25 seems best for getting more single nuclei
markers = ndi.label(local_maxi)[0]
labels = watershed(-distance, markers, mask=chfp_dil, watershed_line=True)
chfp_ws_mask = skimage.morphology.remove_small_objects(labels, min_size=50)
bokeh.io.show(show_three_ims(chfp_filt_gauss[zoom1], labels[zoom1], chfp_ws_mask[zoom1],
                             titles=['original', 'watershed', 'watershed remove small'], 
                             cmap=[None, colorcet.b_glasbey_hv, colorcet.b_glasbey_hv]))

In [ ]:
# relabel image regions
label_image_chfp = skimage.measure.label(chfp_ws_mask)

In [ ]:
props_chfp = skimage.measure.regionprops_table(label_image_chfp, intensity_image=chfp_raw, properties=('label',
                                                                                         'centroid',
                                                                                         'area',
                                                                                         'mean_intensity'))
df_chfp = pd.DataFrame(props_chfp)
chfp_area = df_chfp['area'].sum()
print(chfp_area)
df_chfp

In [ ]:
bins = freedman_diaconis_bins(df_chfp['area'])

hv.Histogram(data=np.histogram(df_chfp['area'], bins=bins),
    kdims=['area']
)


In [ ]:
chfp_array = np.zeros_like(label_image_chfp)
max_size = 1000

for i, label in enumerate(df_chfp.loc[df_chfp.loc[:, 'area'] > max_size]['label']):
    x = label_image_chfp == label
    y = x * label
    chfp_array += y

In [ ]:
bokeh.io.show(
    show_three_ims(chfp_filt_gauss, chfp_ws_mask, chfp_array,
                titles=['original', 'watershed', 'large'],
                cmap=[None, colorcet.b_glasbey_hv, colorcet.b_glasbey_hv]))

In [ ]:
chfp_large_sub = label_image_chfp - chfp_array

bokeh.io.show(
    show_three_ims(chfp_filt_gauss, chfp_ws_mask, chfp_large_sub,
                titles=['original', 'watershed', 'large subtract'],
                cmap=[None, colorcet.b_glasbey_hv, colorcet.b_glasbey_hv]))

In [ ]:
print('Percent CFP+ area:', cfp_area/dapi_area)
print('Percent ChFP+ area:', chfp_area/dapi_area)

## Co-localization

To see colocalization, I will create a mask with CFP **or** ChFP. Then, I will use the mask from each channel from earlier (using Otsu's method) to create masks of CFP only or ChFP only. I will find the nuclei that lie within both and calculate a percentage of double-labeled cells (CFP+/ChFP+).

The first thing I will do is create an 'OR' mask and plot the intensity values of each nuclei. 

In [ ]:
bokeh.io.show(show_four_ims(cfp_filt_gauss, cfp_large_sub, chfp_filt_gauss, chfp_large_sub,
                             titles=['cfp', 'cfp', 'chfp', 'chfp'], 
                             cmap=[None, colorcet.b_glasbey_hv, None, colorcet.b_glasbey_hv]))


I will erode with with a disk structuring element of 3 to make sure that I have individual nuclei when I merge the two masks to make an OR mask later on.

In [ ]:
selem = skimage.morphology.disk(3)
cfp_z4_e = skimage.morphology.erosion(cfp_large_sub, selem)
chfp_z4_e = skimage.morphology.erosion(chfp_large_sub, selem)

selem2 = skimage.morphology.disk(0)
cfp_z4_e2 = skimage.morphology.erosion(cfp_large_sub, selem2)
chfp_z4_e2 = skimage.morphology.erosion(chfp_large_sub, selem2)

bokeh.io.show(show_six_ims(cfp_filt_gauss, cfp_z4_e, cfp_z4_e2, chfp_filt_gauss, chfp_z4_e, chfp_z4_e2,
                             titles=['cfp', 'cfp', 'cfp', 'chfp', 'chfp', 'chfp'], 
                             cmap=[None, colorcet.b_glasbey_hv, colorcet.b_glasbey_hv, 
                                   None, colorcet.b_glasbey_hv, colorcet.b_glasbey_hv]))


In [ ]:
c_ch = np.logical_or(cfp_z4_e, chfp_z4_e)

bokeh.io.show(show_three_ims(cfp_z4_e, chfp_z4_e, c_ch, 
                             titles=['cfp erode', 'chfp erode', 'OR'], 
                             cmap=[colorcet.b_glasbey_hv, colorcet.b_glasbey_hv, None]))


In [ ]:
c_ch_label_bins = skimage.measure.label(c_ch)

props = skimage.measure.regionprops_table(c_ch_label_bins, intensity_image=chfp_raw, properties=('label',
                                                                                         'centroid',
                                                                                         'area',
                                                                                         'mean_intensity'))
df_bin = pd.DataFrame(props)

bins = freedman_diaconis_bins(df_bin['area'])

hv.Histogram(data=np.histogram(df_bin['area'], bins=bins),
    kdims=['area']
)

In [ ]:
c_ch_small = skimage.morphology.remove_small_objects(c_ch, min_size=50)

bokeh.io.show(show_three_ims(cfp_z4_e, chfp_z4_e, c_ch_small,
                             titles=['cfp erode', 'chfp erode', 'OR remove small'], 
                             cmap=[colorcet.b_glasbey_hv, colorcet.b_glasbey_hv, None]))


In [ ]:
# relabel image regions
c_ch_label = skimage.measure.label(c_ch_small)

bokeh.io.show(show_three_ims(cfp_filt_gauss, c_ch_small, c_ch_label,
                             titles=['cfp', 'OR small', 'OR labeled'], 
                             cmap=[None, None, colorcet.b_glasbey_hv]))

In [ ]:
props_cfp_co = skimage.measure.regionprops_table(c_ch_label, intensity_image=cfp_raw, properties=('label',
                                                                                         'centroid',
                                                                                         'area',
                                                                                         'mean_intensity'))
df_cfp_co = pd.DataFrame(props_cfp_co)

props_chfp_co = skimage.measure.regionprops_table(c_ch_label, intensity_image=chfp_raw, properties=('label',
                                                                                         'centroid',
                                                                                         'area',
                                                                                         'mean_intensity'))
df_chfp_co = pd.DataFrame(props_chfp_co)

cfp_vals = df_cfp_co['mean_intensity'].values
chfp_vals = df_chfp_co['mean_intensity'].values
data = {'CFP mean intensity': cfp_vals, 'ChFP mean intensity': chfp_vals}
df_co = pd.DataFrame(data)
df_co

In [ ]:
all_plot = hv.Points(
    data=df_co,
    kdims=['CFP mean intensity', 'ChFP mean intensity'],
    vdims=[],
    label='All nuclei'
).opts()
all_plot

In [ ]:
x = df_co['CFP mean intensity'].values.reshape((-1, 1))
y = df_co['ChFP mean intensity'].values
model = linear_model.LinearRegression().fit(x, y)
r_sq = model.score(x, y)
print('R squared value is:', r_sq)

In [ ]:
cfp_thresh = threshold2 * 65535
chfp_thresh = threshold3 * 65535
print(cfp_thresh, chfp_thresh)

In [ ]:
df_cfp_chfp = df_co.loc[(df_co.loc[:, 'CFP mean intensity'] > cfp_thresh) &
                          (df_co.loc[:, 'ChFP mean intensity'] > chfp_thresh)]
df_cfp = df_co.loc[(df_co.loc[:, 'CFP mean intensity'] > cfp_thresh) &
                          (df_co.loc[:, 'ChFP mean intensity'] < chfp_thresh)]
df_chfp = df_co.loc[(df_co.loc[:, 'CFP mean intensity'] < cfp_thresh) &
                          (df_co.loc[:, 'ChFP mean intensity'] > chfp_thresh)]

In [ ]:
all_plot = hv.Points(
    data=df_co,
    kdims=['CFP mean intensity', 'ChFP mean intensity'],
    vdims=[],
    label='All nuclei'
).opts(color='black', alpha=0.6)

cfp_plot = hv.Points(
    data=df_cfp,
    kdims=['CFP mean intensity', 'ChFP mean intensity'],
    vdims=[],
    label='CFP+ only nuclei'
).opts(color=colorcet.glasbey_dark[3], alpha=0.6)

chfp_plot = hv.Points(
    data=df_chfp,
    kdims=['CFP mean intensity', 'ChFP mean intensity'],
    vdims=[],
    label='ChFP+ only Nuclei'
).opts(color=colorcet.glasbey_dark[13], alpha=0.6)

cfp_chfp_plot = hv.Points(
    data=df_cfp_chfp,
    kdims=['CFP mean intensity', 'ChFP mean intensity'],
    vdims=[],
    label='CFP+/ChFP+ nuclei'
).opts(color=colorcet.glasbey_dark[4], alpha=0.8)

chfp_plot*cfp_plot*cfp_chfp_plot

In [ ]:
all_plot*chfp_plot*cfp_plot*cfp_chfp_plot

In [ ]:
cfp_line = np.linspace(1000, 6000, 200)
chfp_line = np.linspace(550, 2000, 200)

cfp = (cfp_thresh, cfp_line)
chfp = (chfp_line, chfp_thresh)

c = hv.Path(cfp).opts(line_color='black', 
                          line_dash='dotted', 
                          line_alpha=0.5)
ch = hv.Path(chfp).opts(line_color='black', 
                          line_dash='dotted', 
                          line_alpha=0.5)

In [ ]:
chfp_plot*cfp_plot*cfp_chfp_plot*c*ch

In [ ]:
cfp_thresh

In [ ]:
total_cells = len(df_co.index)
co_cells = len(df_cfp_chfp.index)
cfp_only_cells = len(df_cfp.index)
chfp_only_cells = len(df_chfp.index)

In [ ]:
print('Total nuclei:', total_cells)
print('Number of CFP+/ChFP+ cells:', co_cells)
print('Number of CFP+ only cells:', cfp_only_cells)
print('Number of ChFP+ only cells:', chfp_only_cells)

In [ ]:
print('% of CFP+/ChFP+ cells:', co_cells / total_cells * 100)
print('% of CFP+ only cells:', cfp_only_cells / total_cells * 100)
print('% of ChFP+ only cells:', chfp_only_cells / total_cells * 100)

## Correlation of intensity

In [ ]:
'''cfp_thresh = cfp_raw[dapi_filt_gauss_bw]
chfp_thresh = chfp_raw[dapi_filt_gauss_bw]
chfp_bool = np.array(chfp_thresh, dtype=bool)
cfp_filt = cfp_thresh[chfp_bool]
chfp_filt = chfp_thresh[chfp_bool]
data = {'cfp': cfp_filt, 'chfp': chfp_filt}

df = pd.DataFrame(data)
points = hv.Points(
    data=df,
    kdims=['cfp', 'chfp'],
    vdims=[],
)
datashade(points, cmap=process_cmap("viridis", provider="bokeh"), height=600, width=600)
'''

It seems like there are so many points that many are lying on top of each other, preventing the data from being visualized fully because of overplotting. I will use Datashader to plot my data instead. This uses rasterize() to reaggregate the data into a rectangular grid with each grid cell counting the number of points that fall into it. The data is then plotted with a color map.